# LOF

In [2]:
import warnings
warnings.filterwarnings('ignore')

## Libraries import

In [3]:
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report,precision_recall_curve
from sklearn import metrics
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
import time

## Prediction translation

In [4]:
def def_outlier(df):
    if (df['y_pred'] in [-1]):
        val = 1
    else:
        val = 0
    return val

## Arrhythmia

**Dataset source**: http://odds.cs.stonybrook.edu/arrhythmia-dataset/ (data is transformed from .mat to .csv format)

Shebuti Rayana (2016). ODDS Library [http://odds.cs.stonybrook.edu]. Stony Brook, NY: Stony Brook University, Department of Computer Science.

**Additional sources**:

Liu, Fei Tony, Kai Ming Ting, and Zhi-Hua Zhou. “Isolation forest.” 2008 Eighth IEEE International Conference on Data Mining. IEEE, 2008.

K. M. Ting, J. T. S. Chuan, and F. T. Liu. “Mass: A New Ranking Measure for Anomaly Detection.“, IEEE Transactions on Knowledge and Data Engineering, 2009.

F. Keller, E. Muller, K. Bohm.“HiCS: High-contrast subspaces for density-based outlier ranking.” ICDE, 2012.

In [5]:
data = pd.read_csv('./arrhythmia.csv', sep = ',')

In [6]:
# dropping columns that consist only of 0's
data = data.drop(columns = ['Col15', 'Col63', 'Col65', 'Col79', 'Col127', 'Col128','Col135', 'Col137', 'Col139','Col141',
'Col147', 'Col152', 'Col153','Col160','Col200', 'Col260', 'Col270'])

In [7]:
data.head()

,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,Col10,...,Col265,Col266,Col267,Col268,Col269,Col271,Col272,Col273,Col274,y
0,75.0,0.0,190.0,80.0,91.0,193.0,371.0,174.0,121.0,-16.0,...,-0.3,0.0,9.0,-0.9,0.0,0.9,2.9,23.3,49.4,1
1,56.0,1.0,165.0,64.0,81.0,174.0,401.0,149.0,39.0,25.0,...,-0.5,0.0,8.5,0.0,0.0,0.2,2.1,20.4,38.8,0
2,54.0,0.0,172.0,95.0,138.0,163.0,386.0,185.0,102.0,96.0,...,0.9,0.0,9.5,-2.4,0.0,0.3,3.4,12.3,49.0,0
3,55.0,0.0,175.0,94.0,100.0,202.0,380.0,179.0,143.0,28.0,...,0.1,0.0,12.2,-2.2,0.0,0.4,2.6,34.6,61.6,0
4,75.0,0.0,190.0,80.0,88.0,181.0,360.0,177.0,103.0,-16.0,...,-0.4,0.0,13.1,-3.6,0.0,-0.1,3.9,25.4,62.8,1


In [8]:
data.shape

(452, 258)

In [9]:
pd.pivot_table(data,
             values = 'Col1',
               index = 'y', 
              aggfunc = 'count')

,Col1
y,
0,386
1,66


### LOF

In [10]:
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
train_data = min_max_scaler.fit_transform(data.copy())
train_data = pd.DataFrame(train_data)
train_data.columns = data.columns

In [11]:
start = time.process_time()
lof = LocalOutlierFactor(n_neighbors=10, contamination=.1)
y_pred = lof.fit_predict(train_data.loc[:, train_data.columns != 'y'])
y_scores = lof.negative_outlier_factor_
end = time.process_time()
arrhythmia_lof_train_test_time = end - start
print(end - start)

0.859375


In [12]:
train_data['y_pred'] = y_pred
train_data['prediction'] = train_data.apply(def_outlier, axis = 1)
train_data['y_scores'] = -y_scores

In [13]:
confusion_matrix(train_data['y'], train_data['prediction'])

array([[354,  32],
       [ 52,  14]], dtype=int64)

In [14]:
fpr, tpr, _ = metrics.roc_curve(train_data['y'], train_data['y_scores'])
arrhythmia_lof_auc = metrics.auc(fpr, tpr)
metrics.auc(fpr, tpr)

0.7202072538860104

In [15]:
arrhythmia_lof_report = classification_report(train_data['y'], train_data['prediction'], target_names = ['0','1'], output_dict=True)
print(classification_report(train_data['y'], train_data['prediction'], target_names = ['0','1']))

              precision    recall  f1-score   support

           0       0.87      0.92      0.89       386
           1       0.30      0.21      0.25        66

    accuracy                           0.81       452
   macro avg       0.59      0.56      0.57       452
weighted avg       0.79      0.81      0.80       452



In [16]:
precision, recall, thresholds = precision_recall_curve(train_data['y'], train_data['y_scores'])
arrhythmia_lof_auc_precision_recall = metrics.auc(recall, precision)
print(arrhythmia_lof_auc_precision_recall)

0.3377937812666328


## Cardiotocography

**Dataset source**: http://odds.cs.stonybrook.edu/cardiotocogrpahy-dataset/ (data is transformed from .mat to .csv format)

Shebuti Rayana (2016). ODDS Library [http://odds.cs.stonybrook.edu]. Stony Brook, NY: Stony Brook University, Department of Computer Science.

**Additional sources:**

C. C. Aggarwal and S. Sathe, “Theoretical foundations and algorithms for outlier ensembles.” ACM SIGKDD Explorations Newsletter, vol. 17, no. 1, pp. 24–47, 2015.

Saket Sathe and Charu C. Aggarwal. LODES: Local Density meets Spectral Outlier Detection. SIAM Conference on Data Mining, 2016.

In [17]:
data = pd.read_csv('./Cardiotocography.csv')

In [18]:
data.shape

(1831, 22)

In [19]:
data['y'] = data['y'].astype(int)

In [20]:
pd.pivot_table(data,
             values = 'Col1',
               index = 'y', 
              aggfunc = 'count')

,Col1
y,
0,1655
1,176


In [21]:
data.head()

,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,Col10,...,Col13,Col14,Col15,Col16,Col17,Col18,Col19,Col20,Col21,y
0,0.004912,0.693191,-0.203640,0.595322,0.353190,-0.061401,-0.278295,-1.650444,0.759072,-0.420487,...,-0.798376,1.854728,0.622631,0.963083,0.301464,0.193113,0.231498,-0.289786,-0.493294,0
1,0.110729,-0.079903,-0.203640,1.268942,0.396246,-0.061401,-0.278295,-1.710270,0.759072,-0.420487,...,-0.798376,1.854728,0.278625,0.963083,0.301464,0.129265,0.093563,-0.256385,-0.493294,0
2,0.216546,-0.272445,-0.203640,1.050988,0.148753,-0.061401,-0.278295,-1.710270,1.106509,-0.420487,...,-1.332931,0.314688,2.342663,-0.488279,0.061002,0.065417,0.024596,-0.256385,1.140018,0
3,0.004912,0.727346,-0.203640,1.212171,-0.683598,-0.061401,-0.278295,-1.710270,1.106509,-0.420487,...,-1.332931,0.314688,1.654650,-0.488279,0.061002,0.193113,0.093563,-0.323186,1.140018,0
4,-0.100905,0.363595,1.321366,1.027120,0.141359,-0.061401,-0.278295,-0.992364,-0.051613,-0.420487,...,-0.085638,-0.565334,0.278625,-0.488279,-0.059229,0.065417,0.024596,-0.456787,1.140018,0


### LOF

In [22]:
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
train_data = min_max_scaler.fit_transform(data.copy())
train_data = pd.DataFrame(train_data)
train_data.columns = data.columns

In [23]:
# value of n_neighbours is taken as max(10, 0.01*|D|)
start = time.process_time()
lof = LocalOutlierFactor(n_neighbors=18, contamination=.1, novelty=False)
y_pred = lof.fit_predict(train_data.loc[:, train_data.columns != 'y'])
y_scores = lof.negative_outlier_factor_
end = time.process_time()
cardio_lof_train_test_time = end - start 
print(end - start)

0.171875


In [24]:
train_data['y_pred'] = y_pred
train_data['prediction'] = train_data.apply(def_outlier, axis = 1)
train_data['y_scores'] = -y_scores

In [25]:
confusion_matrix(train_data['y'], train_data['prediction'])

array([[1513,  142],
       [ 135,   41]], dtype=int64)

In [26]:
fpr, tpr, _ = metrics.roc_curve(train_data['y'], train_data['y_scores'])
cardio_lof_auc = metrics.auc(fpr, tpr)
metrics.auc(fpr, tpr)

0.6298441362263115

In [27]:
cardio_lof_report = classification_report(train_data['y'], train_data['prediction'], target_names = ['0','1'], output_dict = True)
print(classification_report(train_data['y'], train_data['prediction'], target_names = ['0','1']))

              precision    recall  f1-score   support

           0       0.92      0.91      0.92      1655
           1       0.22      0.23      0.23       176

    accuracy                           0.85      1831
   macro avg       0.57      0.57      0.57      1831
weighted avg       0.85      0.85      0.85      1831



In [28]:
precision, recall, thresholds = precision_recall_curve(train_data['y'], train_data['y_scores'])
cardio_lof_auc_precision_recall = metrics.auc(recall, precision)
print(cardio_lof_auc_precision_recall)

0.18629568436456756


## ForestCover

**Dataset source**: http://odds.cs.stonybrook.edu/forestcovercovertype-dataset/ (data is transformed from .mat to .csv format)

Shebuti Rayana (2016). ODDS Library [http://odds.cs.stonybrook.edu]. Stony Brook, NY: Stony Brook University, Department of Computer Science.

**Additional sources:**

Liu, Fei Tony, Kai Ming Ting, and Zhi-Hua Zhou. “Isolation forest.” 2008 Eighth IEEE International Conference on Data Mining. IEEE, 2008.

K. M. Ting, J. T. S. Chuan, and F. T. Liu. “Mass: A New Ranking Measure for Anomaly Detection.“, IEEE Transactions on Knowledge and Data Engineering, 2009.

Kai Ming Ting, Guang-Tong Zhou, Fei Tony Liu & Tan Swee Chuan. (2010). Mass Estimation and Its Applications. Proceedings of The 16th ACM SIGKDD Conference on Knowledge Discovery and Data Mining 2010. pp. 989-998.

Swee Chuan Tan, Kai Ming Ting & Fei Tony Liu. (2011). Fast Anomaly Detection for Streaming Data. Proceedings of the International Joint Conference on Artificial Intelligence 2011. pp.1151-1156.

In [29]:
data = pd.read_csv('./ForestCover.csv')

In [30]:
data.shape

(286048, 11)

In [31]:
pd.pivot_table(data,
             values = 'Col2',
               index = 'y', 
              aggfunc = 'count')

,Col2
y,
0,283301
1,2747


In [32]:
data.head()

,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,Col10,y
0,2804,139,9,268,65,3180,234,238,135,6121,0
1,2785,155,18,242,118,3090,238,238,122,6211,0
2,2579,132,6,300,-15,67,230,237,140,6031,0
3,2886,151,11,371,26,5253,234,240,136,4051,0
4,2742,134,22,150,69,3215,248,224,92,6091,0


### LOF

In [33]:
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
train_data = min_max_scaler.fit_transform(data.copy())
train_data = pd.DataFrame(train_data)
train_data.columns = data.columns

In [34]:
start = time.process_time()
lof = LocalOutlierFactor(n_neighbors=2860, contamination=.1)
y_pred = lof.fit_predict(train_data.loc[:, train_data.columns != 'y'])
y_scores = lof.negative_outlier_factor_
end = time.process_time()
forestcover_lof_train_test_time = end - start
print(end - start)

827.75


In [35]:
train_data['y_pred'] = y_pred
train_data['prediction'] = train_data.apply(def_outlier, axis = 1)
train_data['y_scores'] = -y_scores

In [36]:
confusion_matrix(train_data['y'], train_data['prediction'])

array([[257323,  25978],
       [   120,   2627]], dtype=int64)

In [37]:
fpr, tpr, _ = metrics.roc_curve(train_data['y'], train_data['y_scores'])
forestcover_lof_auc = metrics.auc(fpr, tpr)
metrics.auc(fpr, tpr)

0.9780153986188571

In [38]:
forestcover_lof_report = classification_report(train_data['y'], train_data['prediction'], target_names = ['0','1'], output_dict = True)
print(classification_report(train_data['y'], train_data['prediction'], target_names = ['0','1']))

              precision    recall  f1-score   support

           0       1.00      0.91      0.95    283301
           1       0.09      0.96      0.17      2747

    accuracy                           0.91    286048
   macro avg       0.55      0.93      0.56    286048
weighted avg       0.99      0.91      0.94    286048



In [39]:
precision, recall, thresholds = precision_recall_curve(train_data['y'], train_data['y_scores'])
forestcover_lof_auc_precision_recall = metrics.auc(recall, precision)
print(forestcover_lof_auc_precision_recall)

0.24822150921582237


## Annthyroid

**Dataset source**: http://odds.cs.stonybrook.edu/annthyroid-dataset/ (data is transformed from .mat to .csv format)

Shebuti Rayana (2016). ODDS Library [http://odds.cs.stonybrook.edu]. Stony Brook, NY: Stony Brook University, Department of Computer Science.

**Aditional sources:**

Abe, Naoki, Bianca Zadrozny, and John Langford. “Outlier detection by active learning.” Proceedings of the 12th ACM SIGKDD international conference on Knowledge discovery and data mining. ACM, 2006.

Liu, Fei Tony, Kai Ming Ting, and Zhi-Hua Zhou. “Isolation forest.” 2008 Eighth IEEE International Conference on Data Mining. IEEE, 2008.

K. M. Ting, J. T. S. Chuan, and F. T. Liu. “Mass: A New Ranking Measure for Anomaly Detection.“, IEEE Transactions on Knowledge and Data Engineering, 2009.

In [40]:
data = pd.read_csv('./annthyroid.csv')

In [41]:
data.head()

,Col1,Col2,Col3,Col4,Col5,Col6,y
0,0.73,0.00060,0.015,0.120,0.082,0.146,0
1,0.24,0.00025,0.030,0.143,0.133,0.108,0
2,0.47,0.00190,0.024,0.102,0.131,0.078,0
3,0.64,0.00090,0.017,0.077,0.090,0.085,0
4,0.23,0.00025,0.026,0.139,0.090,0.153,0


In [42]:
data.shape

(7200, 7)

In [43]:
pd.pivot_table(data,
             values = 'Col1',
               index = 'y', 
              aggfunc = 'count')

,Col1
y,
0,6666
1,534


### LOF

In [44]:
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
train_data = min_max_scaler.fit_transform(data.copy())
train_data = pd.DataFrame(train_data)
train_data.columns = data.columns

In [45]:
start = time.process_time()

lof = LocalOutlierFactor(n_neighbors=72, contamination=.1, novelty=False)
y_pred = lof.fit_predict(train_data.loc[:, train_data.columns != 'y'])
y_scores = lof.negative_outlier_factor_

end = time.process_time()
annthyroid_lof_train_test_time = end - start
print(end - start)

0.671875


In [46]:
train_data['y_pred'] = y_pred
train_data['prediction'] = train_data.apply(def_outlier, axis = 1)
train_data['y_scores'] = -y_scores

In [47]:
confusion_matrix(train_data['y'], train_data['prediction'])

array([[6132,  534],
       [ 348,  186]], dtype=int64)

In [48]:
fpr, tpr, _ = metrics.roc_curve(train_data['y'], train_data['y_scores'])
annthyroid_lof_auc = metrics.auc(fpr, tpr)
metrics.auc(fpr, tpr)

0.7161421198299605

In [49]:
annthyroid_lof_report = classification_report(train_data['y'], train_data['prediction'], target_names = ['0','1'], output_dict = True)
print(classification_report(train_data['y'], train_data['prediction'], target_names = ['0','1']))

              precision    recall  f1-score   support

           0       0.95      0.92      0.93      6666
           1       0.26      0.35      0.30       534

    accuracy                           0.88      7200
   macro avg       0.60      0.63      0.61      7200
weighted avg       0.90      0.88      0.89      7200



In [50]:
precision, recall, thresholds = precision_recall_curve(train_data['y'], train_data['y_scores'])
annthyroid_lof_auc_precision_recall = metrics.auc(recall, precision)
print(annthyroid_lof_auc_precision_recall)

0.20158425964287524


## Credit card

**Dataset source**: https://www.kaggle.com/mlg-ulb/creditcardfraud

**Additional sources:**

Andrea Dal Pozzolo, Olivier Caelen, Reid A. Johnson and Gianluca Bontempi. Calibrating Probability with Undersampling for Unbalanced Classification. In Symposium on Computational Intelligence and Data Mining (CIDM), IEEE, 2015

Dal Pozzolo, Andrea; Caelen, Olivier; Le Borgne, Yann-Ael; Waterschoot, Serge; Bontempi, Gianluca. Learned lessons in credit card fraud detection from a practitioner perspective, Expert systems with applications,41,10,4915-4928,2014, Pergamon

Dal Pozzolo, Andrea; Boracchi, Giacomo; Caelen, Olivier; Alippi, Cesare; Bontempi, Gianluca. Credit card fraud detection: a realistic modeling and a novel learning strategy, IEEE transactions on neural networks and learning systems,29,8,3784-3797,2018,IEEE

Dal Pozzolo, Andrea Adaptive Machine learning for credit card fraud detection ULB MLG PhD thesis (supervised by G. Bontempi)

Carcillo, Fabrizio; Dal Pozzolo, Andrea; Le Borgne, Yann-Aël; Caelen, Olivier; Mazzer, Yannis; Bontempi, Gianluca. Scarff: a scalable framework for streaming credit card fraud detection with Spark, Information fusion,41, 182-194,2018,Elsevier

Carcillo, Fabrizio; Le Borgne, Yann-Aël; Caelen, Olivier; Bontempi, Gianluca. Streaming active learning strategies for real-life credit card fraud detection: assessment and visualization, International Journal of Data Science and Analytics, 5,4,285-300,2018,Springer International Publishing

Bertrand Lebichot, Yann-Aël Le Borgne, Liyun He, Frederic Oblé, Gianluca Bontempi Deep-Learning Domain Adaptation Techniques for Credit Cards Fraud Detection, INNSBDDL 2019: Recent Advances in Big Data and Deep Learning, pp 78-88, 2019

Fabrizio Carcillo, Yann-Aël Le Borgne, Olivier Caelen, Frederic Oblé, Gianluca Bontempi Combining Unsupervised and Supervised Learning in Credit Card Fraud Detection Information Sciences, 2019

Yann-Aël Le Borgne, Gianluca Bontempi Machine Learning for Credit Card Fraud Detection - Practical Handbook

In [51]:
data = pd.read_csv('./creditcard.csv')

In [52]:
data = data.drop(columns = ['Time'])

In [53]:
data.shape

(284807, 30)

In [54]:
data.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [55]:
pd.pivot_table(data,
             values = 'V1',
               index = 'Class', 
              aggfunc = 'count')

,V1
Class,
0,284315
1,492


### LOF

In [56]:
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
train_data = min_max_scaler.fit_transform(data.copy())
train_data = pd.DataFrame(train_data)
train_data.columns = data.columns

In [57]:
start = time.process_time()

lof = LocalOutlierFactor(n_neighbors=2848, contamination=.1, novelty=False)
y_pred = lof.fit_predict(train_data.loc[:, train_data.columns != 'Class'])
y_scores = lof.negative_outlier_factor_

end = time.process_time()
creditcard_lof_train_test_time = end - start
print(end - start)

3076.515625


In [58]:
train_data['y_pred'] = y_pred
train_data['prediction'] = train_data.apply(def_outlier, axis = 1)
train_data['y_scores'] = -y_scores

In [59]:
confusion_matrix(train_data['Class'], train_data['prediction'])

array([[256280,  28035],
       [    46,    446]], dtype=int64)

In [60]:
fpr, tpr, _ = metrics.roc_curve(train_data['Class'], train_data['y_scores'])
creditcard_lof_auc = metrics.auc(fpr, tpr)
metrics.auc(fpr, tpr)

0.9599787265041109

In [61]:
creditcard_lof_report = classification_report(train_data['Class'], train_data['prediction'], target_names = ['0','1'], output_dict = True)
print(classification_report(train_data['Class'], train_data['prediction'], target_names = ['0','1']))

              precision    recall  f1-score   support

           0       1.00      0.90      0.95    284315
           1       0.02      0.91      0.03       492

    accuracy                           0.90    284807
   macro avg       0.51      0.90      0.49    284807
weighted avg       1.00      0.90      0.95    284807



In [62]:
precision, recall, thresholds = precision_recall_curve(train_data['Class'], train_data['y_scores'])
creditcard_lof_auc_precision_recall = metrics.auc(recall, precision)
print(creditcard_lof_auc_precision_recall)

0.452637682764581


## Mammography

**Dataset source**: http://odds.cs.stonybrook.edu/mammography-dataset/ (data is transformed from .mat to .csv format)

Shebuti Rayana (2016). ODDS Library [http://odds.cs.stonybrook.edu]. Stony Brook, NY: Stony Brook University, Department of Computer Science.

**Additional sources:**

Abe, Naoki, Bianca Zadrozny, and John Langford. “Outlier detection by active learning.” Proceedings of the 12th ACM SIGKDD international conference on Knowledge discovery and data mining. ACM, 2006.

Liu, Fei Tony, Kai Ming Ting, and Zhi-Hua Zhou. “Isolation forest.” 2008 Eighth IEEE International Conference on Data Mining. IEEE, 2008.

K. M. Ting, J. T. S. Chuan, and F. T. Liu. “Mass: A New Ranking Measure for Anomaly Detection.“, IEEE Transactions on Knowledge and Data Engineering, 2009.

In [63]:
data = pd.read_csv('./mammography.csv')

In [64]:
data.head()

,Col1,Col2,Col3,Col4,Col5,Col6,y
0,0.230020,5.072578,-0.276061,0.832444,-0.377866,0.480322,0
1,0.155491,-0.169390,0.670652,-0.859553,-0.377866,-0.945723,0
2,-0.784415,-0.443654,5.674705,-0.859553,-0.377866,-0.945723,0
3,0.546088,0.131415,-0.456387,-0.859553,-0.377866,-0.945723,0
4,-0.102987,-0.394994,-0.140816,0.979703,-0.377866,1.013566,0


In [65]:
pd.pivot_table(data,
             values = 'Col1',
               index = 'y', 
              aggfunc = 'count')

,Col1
y,
0,10923
1,260


In [66]:
data.shape

(11183, 7)

### LOF

In [67]:
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
train_data = min_max_scaler.fit_transform(data.copy())
train_data = pd.DataFrame(train_data)
train_data.columns = data.columns

In [68]:
start = time.process_time()

lof = LocalOutlierFactor(n_neighbors=111, contamination=.1, novelty=False)
y_pred = lof.fit_predict(train_data.loc[:, train_data.columns != 'y'])
y_scores = lof.negative_outlier_factor_

end = time.process_time()
mammography_lof_train_test_time = end - start
print(end - start)

1.15625


In [69]:
train_data['y_pred'] = y_pred
train_data['prediction'] = train_data.apply(def_outlier, axis = 1)
train_data['y_scores'] = -y_scores

In [70]:
confusion_matrix(train_data['y'], train_data['prediction'])

array([[9941,  982],
       [ 123,  137]], dtype=int64)

In [71]:
fpr, tpr, _ = metrics.roc_curve(train_data['y'], train_data['y_scores'])
mammography_lof_auc = metrics.auc(fpr, tpr)
metrics.auc(fpr, tpr)

0.8587715406446524

In [72]:
mammography_lof_report = classification_report(train_data['y'], train_data['prediction'], target_names = ['0','1'], output_dict = True)
print(classification_report(train_data['y'], train_data['prediction'], target_names = ['0','1']))

              precision    recall  f1-score   support

           0       0.99      0.91      0.95     10923
           1       0.12      0.53      0.20       260

    accuracy                           0.90     11183
   macro avg       0.56      0.72      0.57     11183
weighted avg       0.97      0.90      0.93     11183



In [73]:
precision, recall, thresholds = precision_recall_curve(train_data['y'], train_data['y_scores'])
mammography_lof_auc_precision_recall = metrics.auc(recall, precision)
print(mammography_lof_auc_precision_recall)

0.12627233589595446


## Shuttle

**Dataset source**: http://odds.cs.stonybrook.edu/shuttle-dataset/ (data is transformed from .mat to .csv format)

Shebuti Rayana (2016). ODDS Library [http://odds.cs.stonybrook.edu]. Stony Brook, NY: Stony Brook University, Department of Computer Science.

**Additional sources:**

Abe, Naoki, Bianca Zadrozny, and John Langford. “Outlier detection by active learning.” Proceedings of the 12th ACM SIGKDD international conference on Knowledge discovery and data mining. ACM, 2006.

Liu, Fei Tony, Kai Ming Ting, and Zhi-Hua Zhou. “Isolation forest.” 2008 Eighth IEEE International Conference on Data Mining. IEEE, 2008.

K. M. Ting, J. T. S. Chuan, and F. T. Liu. “Mass: A New Ranking Measure for Anomaly Detection.“, IEEE Transactions on Knowledge and Data Engineering, 2009.

Kai Ming Ting, Guang-Tong Zhou, Fei Tony Liu & Tan Swee Chuan. (2010). Mass Estimation and Its Applications. Proceedings of The 16th ACM SIGKDD Conference on Knowledge Discovery and Data Mining 2010. pp. 989-998.

Swee Chuan Tan, Kai Ming Ting & Fei Tony Liu. (2011). Fast Anomaly Detection for Streaming Data. Proceedings of the International Joint Conference on Artificial Intelligence 2011. pp.1151-1156.

In [74]:
data = pd.read_csv('./shuttle.csv', sep = ',')

In [75]:
data.head()

,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,y
0,50,21,77,0,28,0,27,48,22,1
1,53,0,82,0,52,-5,29,30,2,0
2,37,0,76,0,28,18,40,48,8,0
3,37,0,79,0,34,-26,43,46,2,0
4,85,0,88,-4,6,1,3,83,80,1


In [76]:
data.shape

(49097, 10)

In [77]:
pd.pivot_table(data,
             values = 'Col1',
               index = 'y', 
              aggfunc = 'count')

,Col1
y,
0,45586
1,3511


### LOF

In [78]:
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
train_data = min_max_scaler.fit_transform(data.copy())
train_data = pd.DataFrame(train_data)
train_data.columns = data.columns

In [79]:
start = time.process_time()

lof = LocalOutlierFactor(n_neighbors=491, contamination=.1, novelty=False)
y_pred = lof.fit_predict(train_data.loc[:, train_data.columns != 'y'])
y_scores = lof.negative_outlier_factor_

end = time.process_time()
shuttle_lof_train_test_time = end - start
print(end - start)

13.5625


In [80]:
train_data['y_pred'] = y_pred
train_data['prediction'] = train_data.apply(def_outlier, axis = 1)
train_data['y_scores'] = -y_scores

In [81]:
confusion_matrix(train_data['y'], train_data['prediction'])

array([[41343,  4243],
       [ 2844,   667]], dtype=int64)

In [82]:
fpr, tpr, _ = metrics.roc_curve(train_data['y'], train_data['y_scores'])
shuttle_lof_auc = metrics.auc(fpr, tpr)
metrics.auc(fpr, tpr)

0.5099668080049211

In [83]:
shuttle_lof_report = classification_report(train_data['y'], train_data['prediction'], target_names = ['0','1'], output_dict = True)
print(classification_report(train_data['y'], train_data['prediction'], target_names = ['0','1']))

              precision    recall  f1-score   support

           0       0.94      0.91      0.92     45586
           1       0.14      0.19      0.16      3511

    accuracy                           0.86     49097
   macro avg       0.54      0.55      0.54     49097
weighted avg       0.88      0.86      0.87     49097



In [84]:
precision, recall, thresholds = precision_recall_curve(train_data['y'], train_data['y_scores'])
shuttle_lof_auc_precision_recall = metrics.auc(recall, precision)
print(shuttle_lof_auc_precision_recall)

0.1373105149137827


## mnist

**Dataset source**: http://odds.cs.stonybrook.edu/mnist-dataset/ (data is transformed from .mat to .csv format)

Shebuti Rayana (2016). ODDS Library [http://odds.cs.stonybrook.edu]. Stony Brook, NY: Stony Brook University, Department of Computer Science.

**Additional sources:**

Bandaragoda, Tharindu R., et al. “Efficient Anomaly Detection by Isolation Using Nearest Neighbour Ensemble.” 2014 IEEE International Conference on Data Mining Workshop. IEEE, 2014.

In [85]:
data = pd.read_csv('./mnist.csv')

In [86]:
data = data.drop(columns = ['Col1','Col4', 'Col7', 'Col22', 'Col27', 'Col29', 'Col38', 'Col41', 'Col51', 'Col53', 'Col54', 'Col61', 'Col62', 'Col71', 'Col73', 'Col79', 'Col87', 'Col88', 'Col89', 'Col90',
'Col92', 'Col100'])

In [87]:
pd.pivot_table(data,
             values = 'Col2',
               index = 'y', 
              aggfunc = 'count')

,Col2
y,
0,6903
1,700


In [88]:
data.shape

(7603, 79)

In [89]:
data.head()

,Col2,Col3,Col5,Col6,Col8,Col9,Col10,Col11,Col12,Col13,...,Col86,Col91,Col93,Col94,Col95,Col96,Col97,Col98,Col99,y
0,-73.804153,198.205963,-13.124617,-1.1501,-0.141633,179.249390,114.661163,-80.736702,130.659348,162.649841,...,-15.392716,188.055649,-4.469967,158.381409,-137.100632,27.131416,-2.274633,-0.00065,-12.351267,0
1,-73.804153,197.205963,-13.124617,-1.1501,-0.141633,179.249390,-44.338833,-80.736702,128.659348,190.649841,...,-15.392716,186.055649,-4.469967,123.381416,-137.100632,157.131409,-2.274633,-0.00065,-12.351267,0
2,-73.804153,-53.794033,-13.124617,-1.1501,-0.141633,-73.750618,-44.338833,170.263306,130.659348,46.649849,...,-15.392716,188.055649,-4.469967,157.381409,-137.100632,-93.868584,-2.274633,-0.00065,-12.351267,0
3,-73.804153,86.205963,-13.124617,-1.1501,-0.141633,76.249382,208.661163,107.263298,130.659348,190.649841,...,-15.392716,188.055649,-4.469967,157.381409,-137.100632,74.131416,-2.274633,-0.00065,-12.351267,0
4,-27.804153,199.205963,-13.124617,-1.1501,-0.141633,179.249390,-44.338833,-80.736702,130.659348,91.649849,...,-15.392716,188.055649,-4.469967,22.381416,-137.100632,159.131409,-2.274633,-0.00065,-12.351267,0


### LOF

In [90]:
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
train_data = min_max_scaler.fit_transform(data.copy())
train_data = pd.DataFrame(train_data)
train_data.columns = data.columns

In [91]:
start = time.process_time()

lof = LocalOutlierFactor(n_neighbors=76, contamination=.1, novelty=False)
y_pred = lof.fit_predict(train_data.loc[:, train_data.columns != 'y'])
y_scores = lof.negative_outlier_factor_

end = time.process_time()
mnist_lof_train_test_time = end - start
print(end - start)

3.328125


In [92]:
train_data['y_pred'] = y_pred
train_data['prediction'] = train_data.apply(def_outlier, axis = 1)
train_data['y_scores'] = -y_scores

In [93]:
confusion_matrix(train_data['y'], train_data['prediction'])

array([[6403,  500],
       [ 439,  261]], dtype=int64)

In [94]:
fpr, tpr, _ = metrics.roc_curve(train_data['y'], train_data['y_scores'])
mnist_lof_auc = metrics.auc(fpr, tpr)
metrics.auc(fpr, tpr)

0.7818577844001573

In [95]:
mnist_lof_report = classification_report(train_data['y'], train_data['prediction'], target_names = ['0','1'], output_dict = True)
print(classification_report(train_data['y'], train_data['prediction'], target_names = ['0','1']))

              precision    recall  f1-score   support

           0       0.94      0.93      0.93      6903
           1       0.34      0.37      0.36       700

    accuracy                           0.88      7603
   macro avg       0.64      0.65      0.64      7603
weighted avg       0.88      0.88      0.88      7603



In [96]:
precision, recall, thresholds = precision_recall_curve(train_data['y'], train_data['y_scores'])
mnist_lof_auc_precision_recall = metrics.auc(recall, precision)
print(mnist_lof_auc_precision_recall)

0.31325975583437815


## vowels

**Dataset source**: http://odds.cs.stonybrook.edu/japanese-vowels-data/

Shebuti Rayana (2016). ODDS Library [http://odds.cs.stonybrook.edu]. Stony Brook, NY: Stony Brook University, Department of Computer Science.

**Additional sources:**

C. C. Aggarwal and S. Sathe, “Theoretical foundations and algorithms for outlier ensembles.” ACM SIGKDD Explorations Newsletter, vol. 17, no. 1, pp. 24–47, 2015.

Saket Sathe and Charu C. Aggarwal. LODES: Local Density meets Spectral Outlier Detection. SIAM Conference on Data Mining, 2016.

In [97]:
data = pd.read_csv('./vowels.csv')

In [98]:
pd.pivot_table(data,
             values = 'Col1',
               index = 'y', 
              aggfunc = 'count')

,Col1
y,
0.0,1406
1.0,50


In [99]:
data.shape

(1456, 13)

In [100]:
data.head()

,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,Col10,Col11,Col12,y
0,0.580469,-0.902534,0.617899,-0.997942,-2.463799,-0.846455,2.349849,0.375400,-0.649334,1.604637,-0.623060,-0.383125,0.0
1,0.784375,-1.077366,0.615781,-0.921911,-2.388553,-0.638047,2.106684,0.361018,-0.714317,1.260236,-0.423339,-0.287791,0.0
2,0.791292,-1.086242,0.669773,-0.806112,-2.260781,-0.538491,2.053282,0.266492,-0.842815,1.081797,-0.267201,-0.172203,0.0
3,1.217306,-1.083425,0.855483,-0.724879,-2.155552,-0.101879,1.768597,0.303151,-1.044710,0.655290,0.214298,-0.341840,0.0
4,1.065352,-1.030178,0.773297,-0.452289,-1.955907,0.248205,1.530474,0.253740,-0.968961,-0.208287,0.331578,0.007288,0.0


### LOF

In [101]:
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
train_data = min_max_scaler.fit_transform(data.copy())
train_data = pd.DataFrame(train_data)
train_data.columns = data.columns

In [102]:
start = time.process_time()

lof = LocalOutlierFactor(n_neighbors=15, contamination=.1, novelty=False)
y_pred = lof.fit_predict(train_data.loc[:, train_data.columns != 'y'])
y_scores = lof.negative_outlier_factor_

end = time.process_time()
vowels_lof_train_test_time = end - start
print(end - start)

0.046875


In [103]:
train_data['y_pred'] = y_pred
train_data['prediction'] = train_data.apply(def_outlier, axis = 1)
train_data['y_scores'] = -y_scores

In [104]:
confusion_matrix(train_data['y'], train_data['prediction'])

array([[1301,  105],
       [   9,   41]], dtype=int64)

In [105]:
fpr, tpr, _ = metrics.roc_curve(train_data['y'], train_data['y_scores'])
vowels_lof_auc = metrics.auc(fpr, tpr)
metrics.auc(fpr, tpr)

0.9484779516358464

In [106]:
vowels_lof_report = classification_report(train_data['y'], train_data['prediction'], target_names = ['0','1'], output_dict = True)
print(classification_report(train_data['y'], train_data['prediction'], target_names = ['0','1']))

              precision    recall  f1-score   support

           0       0.99      0.93      0.96      1406
           1       0.28      0.82      0.42        50

    accuracy                           0.92      1456
   macro avg       0.64      0.87      0.69      1456
weighted avg       0.97      0.92      0.94      1456



In [107]:
precision, recall, thresholds = precision_recall_curve(train_data['y'], train_data['y_scores'])
vowels_lof_auc_precision_recall = metrics.auc(recall, precision)
print(vowels_lof_auc_precision_recall)

0.2893360153648894


## Seismic

**Dataset source**: http://odds.cs.stonybrook.edu/seismic-dataset/ (data is transformed from .arff to .csv format)

Shebuti Rayana (2016). ODDS Library [http://odds.cs.stonybrook.edu]. Stony Brook, NY: Stony Brook University, Department of Computer Science.

**Additional sources:**

Saket Sathe and Charu C. Aggarwal. LODES: Local Density meets Spectral Outlier Detection. SIAM Conference on Data Mining, 2016.

In [108]:
data = pd.read_csv('./seismic.csv', sep = ',')

In [109]:
data = data.drop(columns = ['nbumps6','nbumps7','nbumps89'])

In [110]:
data.shape

(2584, 16)

In [111]:
drop_enc = OneHotEncoder(drop='first').fit_transform(data[['seismic','seismoacoustic','shift','ghazard']])

In [112]:
cat_var = pd.DataFrame(drop_enc.toarray())
cat_var.columns = ['seismic: b', 'seismoacoustic: b','seismoacoustic: c','shift: W','ghazard: b','ghazard: c']

In [113]:
data = pd.concat([data, cat_var], axis = 1)

In [114]:
data = data.drop(columns = ['seismic','seismoacoustic','shift','ghazard'])

In [115]:
data.head()

,genergy,gpuls,gdenergy,gdpuls,nbumps,nbumps2,nbumps3,nbumps4,nbumps5,energy,maxenergy,class,seismic: b,seismoacoustic: b,seismoacoustic: c,shift: W,ghazard: b,ghazard: c
0,15180,48,-72,-72,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,14720,33,-70,-79,1,0,1,0,0,2000,2000,0,0.0,0.0,0.0,0.0,0.0,0.0
2,8050,30,-81,-78,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,28820,171,-23,40,1,0,1,0,0,3000,3000,0,0.0,0.0,0.0,0.0,0.0,0.0
4,12640,57,-63,-52,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [116]:
pd.pivot_table(data,
             values = 'genergy',
               index = 'class', 
              aggfunc = 'count')

,genergy
class,
0,2414
1,170


### LOF

In [117]:
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
train_data = min_max_scaler.fit_transform(data.copy())
train_data = pd.DataFrame(train_data)
train_data.columns = data.columns

In [118]:
start = time.process_time()
lof = LocalOutlierFactor(n_neighbors=26, contamination=.1)
y_pred = lof.fit_predict(train_data.loc[:, train_data.columns != 'class'])
y_scores = lof.negative_outlier_factor_
end = time.process_time()
seismic_lof_train_test_time = end - start
print(end - start)

0.265625


In [119]:
train_data['y_pred'] = y_pred
train_data['prediction'] = train_data.apply(def_outlier, axis = 1)
train_data['y_scores'] = -y_scores

In [120]:
confusion_matrix(train_data['class'], train_data['prediction'])

array([[2175,  239],
       [ 150,   20]], dtype=int64)

In [121]:
fpr, tpr, _ = metrics.roc_curve(train_data['class'], train_data['y_scores'])
seismic_lof_auc = metrics.auc(fpr, tpr)
metrics.auc(fpr, tpr)

0.5828134899361567

In [122]:
seismic_lof_report = classification_report(train_data['class'], train_data['prediction'], target_names = ['0','1'], output_dict=True)
print(classification_report(train_data['class'], train_data['prediction'], target_names = ['0','1']))

              precision    recall  f1-score   support

           0       0.94      0.90      0.92      2414
           1       0.08      0.12      0.09       170

    accuracy                           0.85      2584
   macro avg       0.51      0.51      0.51      2584
weighted avg       0.88      0.85      0.86      2584



In [123]:
precision, recall, thresholds = precision_recall_curve(train_data['class'], train_data['y_scores'])
seismic_lof_auc_precision_recall = metrics.auc(recall, precision)
print(seismic_lof_auc_precision_recall)

0.07978511311035733


## Musk

**Dataset source**: http://odds.cs.stonybrook.edu/musk-dataset/ (data is transformed from .mat to .csv format)

Shebuti Rayana (2016). ODDS Library [http://odds.cs.stonybrook.edu]. Stony Brook, NY: Stony Brook University, Department of Computer Science.

**Additional sources:**

C. C. Aggarwal and S. Sathe, “Theoretical foundations and algorithms for outlier ensembles.” ACM SIGKDD Explorations Newsletter, vol. 17, no. 1, pp. 24–47, 2015.

In [124]:
data = pd.read_csv('./musk.csv', sep = ',')

In [125]:
data.head()

,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,Col10,...,Col158,Col159,Col160,Col161,Col162,Col163,Col164,Col165,Col166,y
0,46.0,-108.0,-60.0,-69.0,-117.0,49.0,38.0,-161.0,-8.0,5.0,...,-308.0,52.0,-7.0,39.0,126.0,156.0,-50.0,-112.0,96.0,1.0
1,41.0,-188.0,-145.0,22.0,-117.0,-6.0,57.0,-171.0,-39.0,-100.0,...,-59.0,-2.0,52.0,103.0,136.0,169.0,-61.0,-136.0,79.0,1.0
2,46.0,-194.0,-145.0,28.0,-117.0,73.0,57.0,-168.0,-39.0,-22.0,...,-134.0,-154.0,57.0,143.0,142.0,165.0,-67.0,-145.0,39.0,1.0
3,41.0,-188.0,-145.0,22.0,-117.0,-7.0,57.0,-170.0,-39.0,-99.0,...,-60.0,-4.0,52.0,104.0,136.0,168.0,-60.0,-135.0,80.0,1.0
4,41.0,-188.0,-145.0,22.0,-117.0,-7.0,57.0,-170.0,-39.0,-99.0,...,-60.0,-4.0,52.0,104.0,137.0,168.0,-60.0,-135.0,80.0,1.0


In [126]:
data['y'] = data['y'].astype(int)

In [127]:
data.shape

(3062, 167)

In [128]:
pd.pivot_table(data,
             values = 'Col1',
               index = 'y', 
              aggfunc = 'count')

,Col1
y,
0,2965
1,97


### LOF

In [129]:
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
train_data = min_max_scaler.fit_transform(data.copy())
train_data = pd.DataFrame(train_data)
train_data.columns = data.columns

In [130]:
start = time.process_time()
lof = LocalOutlierFactor(n_neighbors=31, contamination=.1)
y_pred = lof.fit_predict(train_data.loc[:, train_data.columns != 'y'])
y_scores = lof.negative_outlier_factor_
end = time.process_time()
musk_lof_train_test_time = end - start
print(end - start)

0.765625


In [131]:
train_data['y_pred'] = y_pred
train_data['prediction'] = train_data.apply(def_outlier, axis = 1)
train_data['y_scores'] = -y_scores

In [132]:
confusion_matrix(train_data['y'], train_data['prediction'])

array([[2661,  304],
       [  94,    3]], dtype=int64)

In [133]:
fpr, tpr, _ = metrics.roc_curve(train_data['y'], train_data['y_scores'])
musk_lof_auc = metrics.auc(fpr, tpr)
metrics.auc(fpr, tpr)

0.3861163748891709

In [134]:
musk_lof_report = classification_report(train_data['y'], train_data['prediction'], target_names = ['0','1'], output_dict=True)
print(classification_report(train_data['y'], train_data['prediction'], target_names = ['0','1']))

              precision    recall  f1-score   support

           0       0.97      0.90      0.93      2965
           1       0.01      0.03      0.01        97

    accuracy                           0.87      3062
   macro avg       0.49      0.46      0.47      3062
weighted avg       0.94      0.87      0.90      3062



In [135]:
precision, recall, thresholds = precision_recall_curve(train_data['y'], train_data['y_scores'])
musk_lof_auc_precision_recall = metrics.auc(recall, precision)
print(musk_lof_auc_precision_recall)

0.025728896376839075


## bank

**Dataset source**: https://github.com/GuansongPang/ADRepository-Anomaly-detection-datasets/tree/main/categorical%20data

Pang, G., Shen, C., Cao, L., & Hengel, A. V. D. (2021). Deep learning for anomaly detection: A review. ACM Computing Surveys (CSUR), 54(2), 1-38.

In [136]:
data = pd.read_csv('./bank.csv')

In [137]:
data.head()

,age,job=housemaid,job=services,job=admin.,job=blue-collar,job=technician,job=retired,job=management,job=unemployed,job=self-employed,...,previous,poutcome=nonexistent,poutcome=failure,poutcome=success,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,class
0,0.209877,0,0,0,0,0,0,0,0,0,...,0.000000,1,0,0,1.000000,0.882307,0.376569,0.980730,1.000000,0
1,0.296296,0,0,1,0,0,0,0,0,0,...,0.000000,1,0,0,1.000000,0.484412,0.615063,0.981183,1.000000,0
2,0.246914,1,0,0,0,0,0,0,0,0,...,0.000000,1,0,0,0.937500,0.698753,0.602510,0.957379,0.859735,0
3,0.160494,0,1,0,0,0,0,0,0,0,...,0.142857,0,1,0,0.333333,0.269680,0.192469,0.150759,0.512287,0
4,0.530864,0,0,0,1,0,0,0,0,0,...,0.000000,1,0,0,0.333333,0.340608,0.154812,0.174790,0.512287,1


In [138]:
pd.pivot_table(data,
             values = 'age',
               index = 'class', 
              aggfunc = 'count')

,age
class,
0,36548
1,4640


### LOF

In [139]:
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
train_data = min_max_scaler.fit_transform(data.copy())
train_data = pd.DataFrame(train_data)
train_data.columns = data.columns

In [140]:
start = time.process_time()
lof = LocalOutlierFactor(n_neighbors=412, contamination=.1)
y_pred = lof.fit_predict(train_data.loc[:, train_data.columns != 'class'])
y_scores = lof.negative_outlier_factor_
end = time.process_time()
bank_lof_train_test_time = end - start
print(end - start)

52.828125


In [141]:
train_data['y_pred'] = y_pred
train_data['prediction'] = train_data.apply(def_outlier, axis = 1)
train_data['y_scores'] = -y_scores

In [142]:
confusion_matrix(train_data['class'], train_data['prediction'])

array([[33373,  3175],
       [ 3696,   944]], dtype=int64)

In [143]:
fpr, tpr, _ = metrics.roc_curve(train_data['class'], train_data['y_scores'])
bank_lof_auc = metrics.auc(fpr, tpr)
metrics.auc(fpr, tpr)

0.6685906795220644

In [144]:
bank_lof_report = classification_report(train_data['class'], train_data['prediction'], target_names = ['0','1'], output_dict=True)
print(classification_report(train_data['class'], train_data['prediction'], target_names = ['0','1']))

              precision    recall  f1-score   support

           0       0.90      0.91      0.91     36548
           1       0.23      0.20      0.22      4640

    accuracy                           0.83     41188
   macro avg       0.56      0.56      0.56     41188
weighted avg       0.82      0.83      0.83     41188



In [145]:
precision, recall, thresholds = precision_recall_curve(train_data['class'], train_data['y_scores'])
bank_lof_auc_precision_recall = metrics.auc(recall, precision)
print(bank_lof_auc_precision_recall)

0.1904556554974769


## Performance

In [146]:
performance = pd.DataFrame(columns = ['F1 score', 'recall', 'precision', 'AUC', 'AUPRC', 'Total time'])

In [147]:
f1_score_lof = {'arrhythmia':arrhythmia_lof_report['1']['f1-score'],
                       'cardio':cardio_lof_report['1']['f1-score'], 
                        'forestcover':forestcover_lof_report['1']['f1-score'], 
                       'annthyroid':annthyroid_lof_report['1']['f1-score'],       
                        'creditcard':creditcard_lof_report['1']['f1-score'], 
                       'mammography':mammography_lof_report['1']['f1-score'], 
                        'shuttle':shuttle_lof_report['1']['f1-score'], 
                      'mnist':mnist_lof_report['1']['f1-score'], 
                  'vowels':vowels_lof_report['1']['f1-score'], 
                  'seismic':seismic_lof_report['1']['f1-score'], 
                  'musk':musk_lof_report['1']['f1-score'], 
                  'bank':bank_lof_report['1']['f1-score'], }
f1_score_lof_df = pd.DataFrame.from_dict(f1_score_lof, orient='index', columns = ['F1 score']).reset_index()

In [148]:
recall_lof = {'arrhythmia':arrhythmia_lof_report['1']['recall'],
                       'cardio':cardio_lof_report['1']['recall'], 
                        'forestcover':forestcover_lof_report['1']['recall'], 
                       'annthyroid':annthyroid_lof_report['1']['recall'],       
                        'creditcard':creditcard_lof_report['1']['recall'], 
                       'mammography':mammography_lof_report['1']['recall'], 
                        'shuttle':shuttle_lof_report['1']['recall'], 
                      'mnist':mnist_lof_report['1']['recall'], 
                  'vowels':vowels_lof_report['1']['recall'], 
                  'seismic':seismic_lof_report['1']['recall'], 
                  'musk':musk_lof_report['1']['recall'], 
                  'bank':bank_lof_report['1']['recall'], }
recall_lof_df = pd.DataFrame.from_dict(recall_lof, orient='index', columns = ['Recall']).reset_index()

In [149]:
precision_lof = {'arrhythmia':arrhythmia_lof_report['1']['precision'],
                       'cardio':cardio_lof_report['1']['precision'], 
                        'forestcover':forestcover_lof_report['1']['precision'], 
                       'annthyroid':annthyroid_lof_report['1']['precision'],       
                        'creditcard':creditcard_lof_report['1']['precision'], 
                       'mammography':mammography_lof_report['1']['precision'], 
                        'shuttle':shuttle_lof_report['1']['precision'], 
                      'mnist':mnist_lof_report['1']['precision'], 
                  'vowels':vowels_lof_report['1']['precision'], 
                  'seismic':seismic_lof_report['1']['precision'], 
                  'musk':musk_lof_report['1']['precision'], 
                  'bank':bank_lof_report['1']['precision'], }
precision_lof_df = pd.DataFrame.from_dict(precision_lof, orient='index', columns = ['Precision']).reset_index()

In [150]:
auc_lof = {'arrhythmia':arrhythmia_lof_auc,
                       'cardio':cardio_lof_auc, 
                        'forestcover':forestcover_lof_auc, 
                       'annthyroid':annthyroid_lof_auc,       
                        'creditcard':creditcard_lof_auc, 
                       'mammography':mammography_lof_auc, 
                        'shuttle':shuttle_lof_auc, 
                      'mnist':mnist_lof_auc, 
                  'vowels':vowels_lof_auc, 
                  'seismic':seismic_lof_auc, 
                  'musk':musk_lof_auc, 
                  'bank':bank_lof_auc}
auc_lof_df = pd.DataFrame.from_dict(auc_lof, orient='index', columns = ['AUC']).reset_index()

In [151]:
auprc_lof = {'arrhythmia':arrhythmia_lof_auc_precision_recall,
                       'cardio':cardio_lof_auc_precision_recall, 
                        'forestcover':forestcover_lof_auc_precision_recall, 
                       'annthyroid':annthyroid_lof_auc_precision_recall,       
                        'creditcard':creditcard_lof_auc_precision_recall, 
                       'mammography':mammography_lof_auc_precision_recall, 
                        'shuttle':shuttle_lof_auc_precision_recall, 
                      'mnist':mnist_lof_auc_precision_recall, 
                  'vowels':vowels_lof_auc_precision_recall, 
                  'seismic':seismic_lof_auc_precision_recall, 
                  'musk':musk_lof_auc_precision_recall, 
                  'bank':bank_lof_auc_precision_recall}
auprc_lof_df = pd.DataFrame.from_dict(auprc_lof, orient='index', columns = ['AUPRC']).reset_index()

In [152]:
total_time_lof = {'arrhythmia':arrhythmia_lof_train_test_time,
                       'cardio':cardio_lof_train_test_time, 
                        'forestcover':forestcover_lof_train_test_time, 
                       'annthyroid':annthyroid_lof_train_test_time,       
                        'creditcard':creditcard_lof_train_test_time, 
                       'mammography':mammography_lof_train_test_time, 
                        'shuttle':shuttle_lof_train_test_time, 
                      'mnist':mnist_lof_train_test_time, 
                  'vowels':vowels_lof_train_test_time, 
                  'seismic':seismic_lof_train_test_time, 
                  'musk':musk_lof_train_test_time, 
                  'bank':bank_lof_train_test_time}
total_time_lof_df = pd.DataFrame.from_dict(total_time_lof, orient='index', columns = ['Total time']).reset_index()

In [153]:
pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(f1_score_lof_df, recall_lof_df, how = 'inner'), precision_lof_df, how ='inner'),
         auc_lof_df, how = 'inner'), auprc_lof_df, how = 'inner'), total_time_lof_df, how = 'inner')

,index,F1 score,Recall,Precision,AUC,AUPRC,Total time
0,arrhythmia,0.250000,0.212121,0.304348,0.720207,0.337794,0.859375
1,cardio,0.228412,0.232955,0.224044,0.629844,0.186296,0.171875
2,forestcover,0.167581,0.956316,0.091837,0.978015,0.248222,827.750000
3,annthyroid,0.296651,0.348315,0.258333,0.716142,0.201584,0.671875
4,creditcard,0.030787,0.906504,0.015660,0.959979,0.452638,3076.515625
5,mammography,0.198695,0.526923,0.122431,0.858772,0.126272,1.156250
6,shuttle,0.158413,0.189974,0.135845,0.509967,0.137311,13.562500
7,mnist,0.357290,0.372857,0.342970,0.781858,0.313260,3.328125
8,vowels,0.418367,0.820000,0.280822,0.948478,0.289336,0.046875
9,seismic,0.093240,0.117647,0.077220,0.582813,0.079785,0.265625
